In [2]:
from abc import ABC


def timeit(f):
    def inner(*args, **kwargs):
        start_time = time.time()
        result = f(*args, **kwargs)
        end_time = time.time()
        #print(end_time - start_time)
        return result, end_time - start_time
    return inner

In [3]:
import time

class Platform:
    def __init__(self, algos, tables):
        self.algos = algos
        self.tables = tables
        self.results = []

    def compress_table(self):
        for algo in self.algos:
            for table in self.tables:
                self.run(algo, table)

    def run(self, algo, table):

        compressed_data, compression_time = algo.compression(table)
        #start_time, compressed_data = algo.compression(table) # start_time in .compression()? why?

        compression_rate = len(compressed_data) / len(table) # TODO: replace len(table) with a proper binary size

        decompressed_data, decompression_time = algo.decompression(compressed_data)
        loss_rate = (table - decompressed_data) / table # TODO: replace with a separate method
        result = {
            "algorithm": algo.__name__,
            "table": table,
            "compression_rate": compression_rate,
            "compression_time": compression_time,
            "decompression_time": decompression_time,
            "loss_rate": loss_rate
        }
        self.results.append(result)

In [19]:
import abc

class CompressionAlgorithm(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def __name__(self):
        ...

    @abc.abstractmethod
    @timeit
    def compression(self, table):
        ...

    @abc.abstractmethod
    @timeit
    def decompression(self, compressed_data):
        ...

class LearningCompressionAlgorithm(CompressionAlgorithm):

    @abc.abstractmethod
    @timeit
    def fit(self, training_data):
        ...

class NonLearningCompressionAlgorithm(CompressionAlgorithm):
    ...

class LosslessCompressionAlgorithm(CompressionAlgorithm):
    ...

class LossyCompressionAlgorithm(CompressionAlgorithm):
    ...

In [24]:
class DummyCompressionAlgorithm(LosslessCompressionAlgorithm):
    __name__ = 'Dummy Compression Algorithm'
    @timeit
    def compression(self, table):
        time.sleep(2)
        return table
    @timeit
    def decompression(self, compressed_data):
        time.sleep(1)
        return compressed_data

In [26]:
class Process(abc.ABC):
    def __init__(self):
        ...
    @property
    @abc.abstractmethod
    def __name__(self):
        ...

    @abc.abstractmethod
    @timeit
    def preprocess(self, raw_table):
        ...

    @abc.abstractmethod
    @timeit
    def postprocess(self, processed_table):
        ...

In [25]:
import numpy as np
platform = Platform([DummyCompressionAlgorithm()], [np.array([1,2,3])])
platform.compress_table()
platform.results

[{'algorithm': 'Dummy Compression Algorithm',
  'table': array([1, 2, 3]),
  'compression_rate': 1.0,
  'compression_time': 2.013608932495117,
  'decompression_time': 1.0040090084075928,
  'loss_rate': array([0., 0., 0.])}]